# Time Series Project

https://automotorsqueretaro.negocio.site/ https://automotorsqueretaro.negocio.site/ https://automotorsqueretaro.negocio.site/ https://automotorsqueretaro.negocio.site/

![](id_for_ideas/logo.png)

https://automotorsqueretaro.negocio.site/ https://automotorsqueretaro.negocio.site/ https://automotorsqueretaro.negocio.site/ https://automotorsqueretaro.negocio.site/

## 2.2 Etapas en el desarrollo de código



Acá se va exponiendo de forma interactiva las etapas técnicas del desarrollo de código 

# 2.2.1 DATA FEED:  Descripción del acondicionamiento del dataset original

In [1]:
import pandas as pd
from paquete_proyecto.herramientas.data_info import data_info
import warnings

warnings.filterwarnings("ignore")

In [2]:
from paquete_proyecto.iniciando.bases import importar_databases, ajustes_finales


ventas, ventas_sin_duplicados = importar_databases()

data = ajustes_finales(ventas)

<div>
    <h4>Inspeccion de cada columna (DESACTUALIZADO)</h4>
    <ol>
        <li> <b>IdCliente</b>: object - código alfanumérico</li>
        <li> <b>NombreCliente</b>: object - string = Nombre, Apellido</li>
        <li> <b>>> Empleado</b>: float - cambiar a int</li>
        <li> <b>Referencia</b>: object: categoría ordinal</li>
        <li> <b>Descripcion</b>: object: categoria cualitativa </li>
        <li> <b>>> CodigoFamilia</b>: id</li>
        <li> <b>Familia</b>: Areas comerciales: son 5 que se dividen en "producto" o "servicio"</li>
        <li> <b>>>Cantidad</b>: object: cambiar a int</li>
        <li> <b>>> Ventas</b>: int: cambiar a float</li>
        <li> <b>Localidad</b>: object: categoria bussines</li>
        <li> <b>>> Area</b>: float: cambiar a int</li>
    </ol>
</div>

## CIERRE ETAPA: 2.2.1 DATA PREPARE

### DATASET: 

#### DESCRIPCION:

- dataset original: con la variable "Fecha" como indice

In [3]:
display(data.sample(3))
display(data_info(data=data, name="ventas"))

,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area
Fecha,,,,,,,,,,,,
2018-12-22,C79980772,ESPINOSA ESTEBAN,48,SE-CS-1-0001,ALINEACION AUTO,106,SERVICIOS,1,37815.0,Usaquen,Santa ana,33
2018-07-27,C860040576,COLTANQUES SAS,2,SE-GF-1-0001,ALINEACION,106,SERVICIOS,1,18900.0,Bogota,Grandes flotas,21
2018-11-05,C830096897,MONTESSORI S A,10,SE-CS-4-0001,REPARACION MENOR DE LLANTA (DESPINCHE ) TACO,106,SERVICIOS,1,10924.0,Engativa,Calle 80,31


info de ventas,columna,type,count,NaN,NaN_pct,unique,unique_pct
0,IdCliente,object,127255,0,0.0 %,24087,18.93 %
1,NombreCliente,object,127255,0,0.0 %,21816,17.14 %
2,Empleado,int32,127255,0,0.0 %,56,0.04 %
3,Referencia,object,127255,0,0.0 %,1854,1.46 %
4,Descripcion,object,127255,0,0.0 %,1814,1.43 %
5,CodigoFamilia,int32,127255,0,0.0 %,5,0.0 %
6,Familia,object,127255,0,0.0 %,5,0.0 %
7,Cantidad,int32,127255,0,0.0 %,212,0.17 %
8,Ventas,float64,127255,0,0.0 %,14279,11.22 %
9,Localidad,object,127255,0,0.0 %,6,0.0 %


-----

# 2.2.2 DATA PREPARE: Tratamiento del dataset de trabajo

    1ro - AÑADIR "Id"

        Añadimos un registro único para cada transacción


    2do- AJUSTAR COTIZACION "Ventas_USD":


        AGREGAR COLUMNA "Ventas_USD" QUE SEA  "Ventas" A LA COTIZACION DEL "USD" DE LA FECHA

    
    3ro- CATEGORIZACION:

        Se crean las siguientes categorías desde la variable "Familia":
            - category_1: binaria (producto, servicio) - categoria: [valores]  - servicio: [SERVICIOS, REENCAUCHE], producto: [LLANTA, FILTRO, LUBRICANTES]
    
        Se crean las siguientes sub-categorías desde la variable "Familia" entrada "SERVICIOS":
            - category_2: categórica (15 categorias en total) - RegEx

    
    4to- COMPLETAR EL DATASET CON LAS FECHAS FALTANTES
    

## 1ro- AÑADIR "Id"

In [4]:
from paquete_proyecto.preprocesamiento.hard_code import hard_category_id

data = hard_category_id(data)

## 2do- AJUSTAR COTIZACION: "Ventas_USD"

son 255 semanas que tomamos cotizacion

In [5]:
from paquete_proyecto.preprocesamiento.forex_api import alpha_vantage_fx_api
from paquete_proyecto.preprocesamiento.cotizacion import (
    criterio_valor_apertura,
    preparar_cotizacion,
    agregar_cotizacion,
)

import os
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
TOKEN = os.environ["TOKEN_AV"]

cotizacion = alpha_vantage_fx_api("FX_WEEKLY", "COP", "USD", TOKEN)
cotizacion = criterio_valor_apertura(preparar_cotizacion(data, cotizacion), "COP/USD")

data = agregar_cotizacion(data, cotizacion)
data.loc[:, "Ventas_USD"] = data["Ventas"] * data["Cotizacion_USD"]

## 3ro- CATEGORIZACION

1ro. Categorizacion binaria "producto", "servicio"


2do. SubCategorizacion RegEx a Familia-SERVICIOS (70% de los registros)

        - Se subcategoriza el registro "SERVICIOS" perteneciente a la variable "Familia"
        - Son 14 etiquetas que surgen por extraccion de patrones RegEx desde la variable "Descripcion"
        - El remamente se categorizó con la etiqueta "otros"


In [7]:
from paquete_proyecto.preprocesamiento.hard_code import hard_category_1

# CATEGORY_1
data = hard_category_1(data)

In [8]:
from paquete_proyecto.preprocesamiento.regex import (
    return_match_id,
    category_apply,
    merge_category,
)

# CATEGORY_2
data_regex_servicios = data.loc[data["Familia"] == "SERVICIOS"].set_index("Id")


patrones = dict(
    balanceo="(BALANCEO)",
    alineacion="(ALINEACION)",
    reparacion="(REPARACION)",
    cambio_de_aceite="(CAMBIO(.?.?.?.?)ACEITE)",
    calibracion="(CALIBRACION)",
    montaje="(MONTAJE)",
    rotacion="(ROTACION)",
    mano_de_obra="(MANO(.?.?.?.?)OBRA)",
    frenos="(AJUSTE(.?.?.?.?)FRENOS|CAMBIO(.?.?.?.?)PASTILLAS|REVISION(.?.?.?.?)FRENOS)",
    rectificacion="(RECTIFICACION)",
    servicios="(SERVICIO(.?))",
    cupon_combo="(CUPON|COMBO)",
    desvare_ruta="(DESVARE(.?.?.?.?.?.?.?.?)RUTA)",
    llanta="((REGRABACION|REMANENTE|INSPECCION|MARCACION|VULCANIZADO)(.?.?.?.?.?.?.?.?.?.?.?.?.?.?.?.?)LLANTA(.?))",
    otros="Patron nulo",
)


indice_categorias = return_match_id(data_regex_servicios, "Descripcion", patrones)
feature_categoria = category_apply(data_regex_servicios, indice_categorias)

data = merge_category(
    data, feature_categoria, "Id", "category_2"
)  # , fillna= 'NaL') # Not a Label

## 4to- Completar Fecha Faltantes

In [9]:
from paquete_proyecto.preprocesamiento.muestreo import complete_dates

data = complete_dates(data)

## CIERRE ETAPA: 2.2.2 DATA PREPARE

### DATASET:

- Se añade "Cotizacion_USD", "Ventas_USD"," category_1", y "category_2"
- Se completan las fechas faltantes

In [10]:
display(data.sample(3))
display(data_info(data=data, name="ventas"))

,Id,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area,Cotizacion_USD,Ventas_USD,category_1,category_2
Fecha,,,,,,,,,,,,,,,,,
2018-05-26,62585,C79399754,PRADA JOSE,14,SE-CS-8-0001,CAMBIO DE ACEITE,106,SERVICIOS,1,2521,Suba,Suba,32,0.00035,0.88235,servicio,cambio_de_aceite
2019-10-04,110662,C860054978,ADISPETROL SA,2,LL-PLC-0163,X MULTI D2 295/80R225,101,LLANTA,16,20588297,Engativa,Ventas externas,22,0.00029,5970.60613,producto,NaL
2019-12-31,120362,C11188069,COY JUAN CARLOS,2,LUA-0030,MOBIL XHP 20W50 GRANEL,103,LUBRICANTES,1,19395,Engativa,Calle 80,31,0.00030,5.81850,producto,NaL


info de ventas,columna,type,count,NaN,NaN_pct,unique,unique_pct
0,Id,int32,127979,0,0.0 %,127256,99.44 %
1,IdCliente,object,127979,0,0.0 %,24088,18.82 %
2,NombreCliente,object,127979,0,0.0 %,21816,17.05 %
3,Empleado,int32,127979,0,0.0 %,57,0.04 %
4,Referencia,object,127979,0,0.0 %,1855,1.45 %
5,Descripcion,object,127979,0,0.0 %,1815,1.42 %
6,CodigoFamilia,int32,127979,0,0.0 %,6,0.0 %
7,Familia,object,127979,0,0.0 %,6,0.0 %
8,Cantidad,int32,127979,0,0.0 %,213,0.17 %
9,Ventas,int32,127979,0,0.0 %,14280,11.16 %


In [11]:
# TODO: En la funcion que crea category_2 implementar un fillna que sea NaL por default

import sys

sys.exit(0)


"""

Este es un mensaje importante:

No tengo un lugar fijo para decantar mis pensamientos:

Recien aplicando data.sample() pude ver que el servicio de REENCAUCHE realizó una venta de 0.15 centavos de dolar, insignificante, y que el servicio SERVICIOS una de 2.9usd tmb es poco.

Estas observaciones no pueden quedar olvidadas

2.
Hay muchos Familias, Descripcion que solo se han vendido 1 sola vez

3. x (!) WRONG
el top 50 de los peores SERVICIOS son REENCAUCHES MAL INGRESADOS

4.
los FILTROS, top top top no reportaron más de 4000 dolares de ganancias en todo el periodo.

5.
las LLANTAS reportan ganancias inmensas, al mismo tiempo que otras reportan poco.
"""

SystemExit: 0

-----

# (III) RESEARCH:
    1- 

    2- INDICADORES BASICOS
    3- ARMANDO VECTORES DE SERIES TEMPORALES (Para Machine Learning)


In [ ]:
# Indicadores Básicos

# 1. total_acumulado

total_acumulado = data.groupby(data.index)[["Cantidad", "Ventas_USD"]].sum()
total_acumulado.loc[:, "tot_acum_Ventas_USD"] = total_acumulado.Ventas_USD.cumsum()
total_acumulado.loc[:, "tot_acum_pct_Ventas_USD"] = round(
    total_acumulado["Ventas_USD"].pct_change() * 100, 2
)
total_acumulado

,Cantidad,Ventas_USD,tot_acum_Ventas_USD,tot_acum_pct_Ventas_USD
Fecha,,,,
2016-01-09,263.0,30933.77640,3.093378e+04,NaN
2016-01-10,152.0,4385.57970,3.531936e+04,-85.82
2016-01-11,420.0,47267.68410,8.258704e+04,977.80
2016-01-12,343.0,26772.69450,1.093597e+05,-43.36
2016-01-13,0.0,0.00000,1.093597e+05,-100.00
...,...,...,...,...
2020-11-28,0.0,0.00000,2.419748e+07,NaN
2020-11-29,0.0,0.00000,2.419748e+07,NaN
2020-11-30,0.0,0.00000,2.419748e+07,NaN


# (IV) REPORT:

In [ ]:
data

,index,Id,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area,Cotizacion_USD,Ventas_USD,category_1,category_2
Fecha,,,,,,,,,,,,,,,,,,
2016-01-09,0,1,C1070956433,PINTO JOHN,24,LL-2R-0038,PILOT STREET 110/80-17,101,LLANTA,1,129310.0,Puente aranda,Calle 13,35,0.00030,38.79300,producto,NaN
2016-01-09,1,2,C19347046,RUIZ LUIS FERNANDO,10,SE-CS-2-0001,BALANCEO AUTO,106,SERVICIOS,4,31035.0,Engativa,Calle 80,31,0.00030,9.31050,servicio,balanceo
2016-01-09,2,3,C800232717,CORSO Y CIA S EN C,10,SE-CS-2-0002,BALANCEO CAMIONETA,106,SERVICIOS,4,31034.0,Engativa,Calle 80,31,0.00030,9.31020,servicio,balanceo
2016-01-09,3,4,C79523685,GONZALEZ JUAN CARLOS,10,SE-CS-2-0002,BALANCEO CAMIONETA,106,SERVICIOS,4,31034.0,Engativa,Calle 80,31,0.00030,9.31020,servicio,balanceo
2016-01-09,4,5,C800214037,EXPRESER SAS,5,REB-0063,REENC XZE1 235/75R175,102,REENCAUCHE,1,29892.0,Engativa,Ventas externas,22,0.00030,8.96760,servicio,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-02,127250,127251,C17050457,GAMBOA PAEZ JOSE MARIO,44,SE-CS-1-0001,ALINEACION AUTO,106,SERVICIOS,1,37815.0,Engativa,Calle 80,31,0.00028,10.58820,servicio,alineacion
2020-12-02,127251,127252,C51590721,DAZA MIRIAM,44,SE-CS-1-0006,ALINEACION BUS,106,SERVICIOS,1,37815.0,Engativa,Calle 80,31,0.00028,10.58820,servicio,alineacion
2020-12-02,127252,127253,C860517112,TRANSPORTES ESPECIALES COLEGIOS Y TURISMO SA,50,SE-CS-1-0005,ALINEACION BUSETA O CAMION LIVIANO,106,SERVICIOS,1,37815.0,Puente aranda,Calle 13,35,0.00028,10.58820,servicio,alineacion


# (V) EXTRAS:

## 2do- PREPARAR DATASETS PARA MUESTREOS TEMPORALES

    2.1- Completar fechas faltante

    2.2- Armar muestreos temporales

    2.3- Visualizar poblacion



**EXPLICACION DEL GRAFICO**<br />
La variable "lista_mensual", es una lista cuyos elementos son datasets. <br />
Son 59 datasets que representan 1790 días en total<br />
Cada dataset representa un período de 30 días. <br />
Cada columna del gráfico representa el "len()" de un dataset. <br />
Algunos períodos tuvieron más cantidad de transacciones que otros. <br />

------DESC------

Primero tenemos que armar cada una de las series temporales

Para poder aplicar Cross-Validation a la serie temporal, es necesario completar con las fechas faltantes con algun valor default en sus variables. De esta forma podremos garantizar que nuestro folds representan el mismo timedelta.

------EJ------

Para hacer cross validation con scikit learn sobre data:

Si empezamos por 2016-01-09 y terminamos en 2020-12-02, tenemos 1790 días. Nuestro "log" es 1 día.

Como querémos cortes de igual tamaño, y sin gap al max_train_size y al test_size le damos el mismo valor.

Si quisieramos cortes bimensuales ingresamos los parámetros con valor 60 (días).

Para n_splits hacemos 1790 / 60 = 29.8 por eso elegimos 29 folds.

In [ ]:
from paquete_proyecto.preprocesamiento.muestreo import complete_dates, timeseries_cv

data_complete = complete_dates(data)

lista_mensual = timeseries_cv(data_complete, 59, 30, 30)


# VISUALIZACION:
# POBLACION: LONGITUD DE LOS DATASETS MUESTREADOS
longitud = []
for elemento in lista_mensual:
    longitud.append(len(elemento.dropna()))

pd.Series(longitud).plot.bar(figsize=(15, 5)).set(
    title="Cantidad de ventas cada 30 días"
)

In [ ]:
# TOP 50 DE LAS FAMILIAS DESCRIPCION MAS VENDIDAS
# CHEQUEAR EL TEMA DE LOS MULTIINDEX LOCO FLACO MACHO CAMPEON DE ARENA SABIO DE MONTAÑA FACHA DE LA EPOCA EJEMPLO DE MARTIR CADAVER DE LUZ
data.groupby(["Familia", "Descripcion"])["Cantidad"].count().to_frame().sort_values(
    by="Familia"
).sort_values(by="Cantidad", ascending=False).head(50)


data.groupby("Familia")["Cantidad"].count()
data.loc[data.Familia == "REENCAUCHE"].groupby(["Familia", "Descripcion"])[
    "Ventas_USD"
].count().to_frame().sort_values(by="Familia").sort_values(
    by="Ventas_USD", ascending=False
)
data.loc[data.Familia == "SERVICIOS"].groupby(["Familia", "Descripcion"])[
    "Ventas_USD"
].sum().to_frame().sort_values(by="Ventas_USD", ascending=False)

Ventas_USD
Familia   Descripcion                                             
SERVICIOS CONTRATO DE SERVICIOS/ COSTO POR KILOMETRO  503113.42385
          CONTRATO DE SERVICIOS /ALPINA               396992.36864
          ALINEACION                                  357223.32995
          CONTRATO DE SERVICIOS / SI-99               274413.98315
          CONTRATO DE SERVICIOS/ SUMA                 234065.08021
...                                                            ...
          REPARACION MENOR OTR                             7.18922
          VULCANIZADO LLANTA                               3.90755
          DESINFECCION DE DUCTOS                           3.07562
          ALQUILER DE EQUIPOS Y MAQUINARIA                 1.99478
          RESTAURACION DE FAROLAS                          1.41176

[130 rows x 1 columns]

Fecha


,Id,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area,category_1,Cotizacion_USD,Ventas_USD,category_2
Fecha,,,,,,,,,,,,,,,,,
2016-01-09,1,C1070956433,PINTO JOHN,24,LL-2R-0038,PILOT STREET 110/80-17,101,LLANTA,1,129310.0,Puente aranda,Calle 13,35,producto,0.00030,38.79300,NaN
2016-01-09,2,C19347046,RUIZ LUIS FERNANDO,10,SE-CS-2-0001,BALANCEO AUTO,106,SERVICIOS,4,31035.0,Engativa,Calle 80,31,servicio,0.00030,9.31050,balanceo
2016-01-09,3,C800232717,CORSO Y CIA S EN C,10,SE-CS-2-0002,BALANCEO CAMIONETA,106,SERVICIOS,4,31034.0,Engativa,Calle 80,31,servicio,0.00030,9.31020,balanceo
2016-01-09,4,C79523685,GONZALEZ JUAN CARLOS,10,SE-CS-2-0002,BALANCEO CAMIONETA,106,SERVICIOS,4,31034.0,Engativa,Calle 80,31,servicio,0.00030,9.31020,balanceo
2016-01-09,5,C800214037,EXPRESER SAS,5,REB-0063,REENC XZE1 235/75R175,102,REENCAUCHE,1,29892.0,Engativa,Ventas externas,22,servicio,0.00030,8.96760,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-02,127251,C17050457,GAMBOA PAEZ JOSE MARIO,44,SE-CS-1-0001,ALINEACION AUTO,106,SERVICIOS,1,37815.0,Engativa,Calle 80,31,servicio,0.00028,10.58820,alineacion
2020-12-02,127252,C51590721,DAZA MIRIAM,44,SE-CS-1-0006,ALINEACION BUS,106,SERVICIOS,1,37815.0,Engativa,Calle 80,31,servicio,0.00028,10.58820,alineacion
2020-12-02,127253,C860517112,TRANSPORTES ESPECIALES COLEGIOS Y TURISMO SA,50,SE-CS-1-0005,ALINEACION BUSETA O CAMION LIVIANO,106,SERVICIOS,1,37815.0,Puente aranda,Calle 13,35,servicio,0.00028,10.58820,alineacion


In [ ]:
"""
Familia - SERVICIOS

ES CAMION? SI O NO?

"""

In [ ]:
# crear una pivot table. que en el indice tenga fechas, en las columnas tenga localidad, y que los valores sean la cantidad de clientes que tuvo.

pd.pivot_table(
    data_complete,
    values="IdCliente",
    index="Fecha",
    columns="Localidad",
    aggfunc="count",
)